In [12]:
#
#This model uses data collected with my scraper "git@github.com:robrmz/Rental-Prices-GDL.git"
#The purpose of this is for the use of a web app that can tell you how much you're expected to pay with certain features.
#For example if you're looking for a 3 bed, 2 bath, 1 parking apartment, it can tell you a $ quantity based on this model. 
#

#explore data with pandas
import pandas as pd

#importing data, initialize dataframe
data = pd.read_csv("data_wrangled.csv")
data = data[["location","price","bedrooms","bathrooms","garage","area"]]
print(data)

#importing label encoder and creating a variable to transform "location" column
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
data['location_fix'] = labelencoder.fit_transform(data['location'])

#Assigning data before split
X = data[["bedrooms", "bathrooms", "garage", "area"]]
y = data[["price"]]



                  location  price  bedrooms  bathrooms  garage  area
0                  Zapopan  22300         3          4       2   180
1            Real Vallarta  12500         2          2       2   100
2         Puertas del Tule  25000         4          4       2   187
3                  Zapopan  58000         3          4       4   700
4        Heroes Nacionales   9000         3          2       2   110
..                     ...    ...       ...        ...     ...   ...
413   Villas del Centinela  10000         2          1       1    95
414  Jardines de Guadalupe  38000         3          3       3   300
415   Parque Metropolitano  19000         3          2       2   108
416               Agrícola  10500         3          3       2   115
417           Nuevo México  14500         3          2       2   119

[418 rows x 6 columns]


In [13]:
from sklearn.model_selection import train_test_split
#Split data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size= 0.8, test_size=0.2, 
                                                      random_state=0)

In [15]:
#metric we're targeting, RMSE
from sklearn.metrics import mean_squared_error

#importing regression methods
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

def get_models():
    #
    rf = RandomForestRegressor(n_estimators=1000, random_state=0)
    xgb_def = xgb.XGBRegressor(n_estimators=301, max_depth=3, learning_rate=0.095)
    lr = LinearRegression()
    svr_def = SVR()
    
    #
    regressors = [rf, xgb_def, lr, svr_def]
    
    #iterating over each method
    for i in regressors:
        i.fit(X_train,y_train)
        i_preds = i.predict(X_valid)
        i_score = mean_squared_error(y_valid, i_preds, squared=False)
        print(str(i) + ": " + str(i_score))
        
get_models()
        

C:\Users\Rob\AppData\Local\Temp/ipykernel_22984/1993285298.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  i.fit(X_train,y_train)


RandomForestRegressor(n_estimators=1000, random_state=0): 13540.87896121794
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.095, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=301, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None): 13442.722540758909
LinearRegression(): 16477.24549947376
SVR(): 26336.062977673362


C:\Users\Rob\anaconda3\envs\DataScience\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#According to the function's output, best model is XGBRegressor, though Random Forest it is pretty close. 
#We're using either, since after tweaking parameters, error difference is marginal. 
#Next step, train full model, turn it into a pickle, build web app. 